In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'n2o': 3.2e-07}
band = [9]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.037020                    1 -0.012154
0.000750        2 -0.036781                    2 -0.012018
0.001052        3 -0.036237                    3 -0.011678
0.001476        4 -0.035653                    4 -0.011326
0.002070        5 -0.035061                    5 -0.010984
0.002904        6 -0.034479                    6 -0.010622
0.004074        7 -0.033927                    7 -0.010267
0.005714        8 -0.033382                    8 -0.009891
0.008015        9 -0.032878                    9 -0.009526
0.011243       10 -0.032419                   10 -0.009139
0.015771       11 -0.032029                   11 -0.008752
0.022122       12 -0.031741                   12 -0.008368
0.031031       13 -0.031600                   13 -0.007974
0.043528       14 -0.031622                   14 -0.007567
0.061057       15 -0.030354                   15 -0.006241
0.085645       16 -0.024805                   16 -0.002211
0.120136       17 -0.014869                   17  0.004680
0.168516       18 -0.002548                   18  0.013471
0.236378       19  0.012299                   19  0.024356
0.331549       20  0.030115                   20  0.037733
0.465100       21  0.051456                   21  0.054095
0.652400       22  0.077133                   22  0.073793
0.915100       23  0.100146                   23  0.091943
1.283650       24  0.097532                   24  0.093068
1.800600       25  0.072587                   25  0.077660
2.525700       26  0.048480                   26  0.059789
3.542800       27  0.030763                   27  0.043513
4.969550       28  0.019154                   28  0.029753
6.970850       29  0.012388                   29  0.018881
9.778100       30  0.008419                   30  0.010598
13.715850      31  0.005363                   31  0.004180
19.239350      32  0.002498                   32 -0.000683
26.987250      33  0.000262                   33 -0.004032
37.855300      34 -0.001606                   34 -0.006712
53.100050      35 -0.004150                   35 -0.010074
73.887500      36 -0.007532                   36 -0.014436
97.662500      37 -0.010719                   37 -0.018628
121.437500     38 -0.013565                   38 -0.022249
145.212500     39 -0.016960                   39 -0.025809
168.987500     40 -0.020367                   40 -0.028854
192.762500     41 -0.022017                   41 -0.030258
216.537500     42 -0.021678                   42 -0.029985
240.312500     43 -0.020868                   43 -0.029047
264.087500     44 -0.019921                   44 -0.027789
287.862500     45 -0.018807                   45 -0.026244
311.637500     46 -0.017505                   46 -0.024444
335.412500     47 -0.016012                   47 -0.022390
359.187500     48 -0.014336                   48 -0.020135
382.962500     49 -0.012488                   49 -0.017686
406.737500     50 -0.010490                   50 -0.015075
430.512500     51 -0.008386                   51 -0.012348
454.287500     52 -0.006222                   52 -0.009555
478.062500     53 -0.004026                   53 -0.006687
501.837500     54 -0.001794                   54 -0.003784
525.612500     55  0.000494                   55 -0.000830
549.387500     56  0.002861                   56  0.002190
573.162500     57  0.005321                   57  0.005285
596.937500     58  0.007884                   58  0.008457
620.712500     59  0.010556                   59  0.011705
644.487500     60  0.013340                   60  0.015024
668.262500     61  0.016241                   61  0.018409
692.037500     62  0.019265                   62  0.021847
715.812500     63  0.022418                   63  0.025378
739.587500     64  0.025703                   64  0.028964
763.362500     65  0.029111                   65  0.032581
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -20.873170  0.000000e+00 -20.873170 -20.697099   
0.000624    2     -20.873174  9.601783e-07 -20.873173 -20.697100   
0.000876    3     -20.873175  1.350753e-06 -20.873174 -20.697101   
0.001229    4     -20.873177  1.910207e-06 -20.873175 -20.697102   
0.001723    5     -20.873180  2.712853e-06 -20.873178 -20.697103   
0.002417    6     -20.873184  3.864185e-06 -20.873180 -20.697104   
0.003391    7     -20.873190  5.515217e-06 -20.873184 -20.697106   
0.004757    8     -20.873198  7.877792e-06 -20.873190 -20.697109   
0.006672    9     -20.873209  1.125737e-05 -20.873197 -20.697114   
0.009359    10    -20.873224  1.608668e-05 -20.873208 -20.697120   
0.013128    11    -20.873245  2.297849e-05 -20.873222 -20.697128   
0.018415    12    -20.873275  3.279705e-05 -20.873242 -20.697139   
0.025830    13    -20.873317  4.675715e-05 -20.873270 -20.697154   
0.036232    14    -20.873376  6.655325e-05 -20.873309 -20.697176   
0.050823    15    -20.873458  9.457458e-05 -20.873364 -20.697206   
0.071291    16    -20.873573  1.359947e-04 -20.873437 -20.697247   
0.100000    17    -20.873726  2.046116e-04 -20.873522 -20.697297   
0.140271    18    -20.873919  3.259716e-04 -20.873593 -20.697352   
0.196760    19    -20.874149  5.387045e-04 -20.873610 -20.697400   
0.275997    20    -20.874401  9.065724e-04 -20.873494 -20.697416   
0.387100    21    -20.874632  1.533633e-03 -20.873098 -20.697351   
0.543100    22    -20.874735  2.588060e-03 -20.872147 -20.697106   
0.761700    23    -20.874480  4.329752e-03 -20.870150 -20.696498   
1.068500    24    -20.873528  7.017835e-03 -20.866510 -20.695288   
1.498800    25    -20.871918  1.037913e-02 -20.861539 -20.693496   
2.102400    26    -20.870073  1.372435e-02 -20.856349 -20.691408   
2.949000    27    -20.868199  1.671186e-02 -20.851487 -20.689186   
4.136600    28    -20.866409  1.924931e-02 -20.847159 -20.686949   
5.802500    29    -20.864816  2.143689e-02 -20.843380 -20.684839   
8.139200    30    -20.863524  2.357361e-02 -20.839951 -20.683038   
11.417000   31    -20.862700  2.601865e-02 -20.836682 -20.681862   
16.014700   32    -20.862787  2.902630e-02 -20.833761 -20.682015   
22.464000   33    -20.864672  3.281952e-02 -20.831852 -20.684770   
31.510500   34    -20.869542  3.796979e-02 -20.831572 -20.692110   
44.200100   35    -20.879223  4.523774e-02 -20.833986 -20.707369   
62.000000   36    -20.897749  5.501245e-02 -20.842737 -20.736961   
85.775000   37    -20.931260  6.731146e-02 -20.863948 -20.789393   
109.550000  38    -20.973931  7.979556e-02 -20.894136 -20.854226   
133.325000  39    -21.025404  9.306562e-02 -20.932338 -20.929984   
157.100000  40    -21.086773  1.066705e-01 -20.980103 -21.016201   
180.875000  41    -21.158802  1.213392e-01 -21.037463 -21.111896   
204.650000  42    -21.239539  1.400705e-01 -21.099468 -21.214504   
228.425000  43    -21.325913  1.653939e-01 -21.160519 -21.321348   
252.200000  44    -21.416558  1.972681e-01 -21.219290 -21.431061   
275.975000  45    -21.510742  2.353481e-01 -21.275394 -21.542816   
299.750000  46    -21.607836  2.794760e-01 -21.328360 -21.655886   
323.525000  47    -21.707237  3.295788e-01 -21.377658 -21.769598   
347.300000  48    -21.808371  3.856168e-01 -21.422754 -21.883348   
371.075000  49    -21.910685  4.475568e-01 -21.463129 -21.996605   
394.850000  50    -22.013654  5.153547e-01 -21.498299 -22.108853   
418.625000  51    -22.116754  5.889110e-01 -21.527843 -22.219643   
442.400000  52    -22.219558  6.680961e-01 -21.551461 -22.328613   
466.175000  53    -22.321722  7.527370e-01 -21.568985 -22.435460   
489.950000  54    -22.422963  8.426408e-01 -21.580323 -22.539882   
513.725000  55    -22.523011  9.376361e-01 -21.585375 -22.641637   
537.500000  56    -22.621579  1.037594e+00 -21.583985 -22.740463   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')